In [148]:
import pandas as pd
import os
from pymilvus import Collection, connections
from pymilvus import connections, utility

import copy

In [149]:
NAMESPACE = 'TESLA_MODEL3'

context_file_root_path = '/Users/yj/Kim/1.work/SKR/8.GenAI/my-small-mechanic/pdf_context'
bm25_model_root_path = '/Users/yj/Kim/1.work/SKR/8.GenAI/my-small-mechanic/vector_db/bm25'
context_text_dir = context_file_root_path + f'/text/{NAMESPACE}'

MILVUS_HOST = 'localhost'
MILVUS_PORT = os.environ['MILVUS_PORT']
COLLECTION_NAME = "HYUNDAI_CAR_MANUAL"

os.makedirs(bm25_model_root_path, exist_ok=True)

connections.connect(alias="default", host=MILVUS_HOST, port=MILVUS_PORT)


In [150]:
if COLLECTION_NAME in utility.list_collections():
    collection = Collection(name=COLLECTION_NAME)
    print("Complete to create collection")

Complete to create collection


In [37]:
M = 20
efConstruction=100

In [38]:
collection.create_index(
    field_name="vector", 
    index_params={
        "index_type": "HNSW", 
        "index_name":"ann_index",
        "metric_type": "COSINE", 
        "params": {"M": M, 'efConstruction':efConstruction} # Cluster 개수
        }
)

Status(code=0, message=)

In [39]:

collection.create_index(
    field_name= "bm25_vector",
    index_params={
        "index_name": "bm25_index",
        "index_type": "SPARSE_INVERTED_INDEX", # the type of index to be created. set to `SPARSE_INVERTED_INDEX` or `SPARSE_WAND`.
        "metric_type": "IP", # the metric type to be used for the index. Currently, only `IP` (Inner Product) is supported.
        "params": {"drop_ratio_build": 0.01}, # the ratio of small vector values to be dropped during indexing.
})

Status(code=0, message=)

In [15]:
collection.load()

# Search

In [151]:
import os
from os import path
import copy

from openai import OpenAI
from pymilvus import AnnSearchRequest
from pymilvus import WeightedRanker, RRFRanker

from pymilvus import Collection, connections
from pymilvus import connections
from pymilvus.model.sparse import BM25EmbeddingFunction
from pymilvus.model.sparse.bm25.tokenizers import build_default_analyzer

In [153]:
analyzer = build_default_analyzer(language="kr")
bm25_ef = BM25EmbeddingFunction(analyzer)
bm25_dir = path.dirname('../bm25/')
bm25_ef.load(bm25_dir + f"/bm25_{NAMESPACE}_params.json")

def convert_sparse(bm25_dict):
    return dict(map(lambda x: (x[0][1], x[1]), bm25_dict.todok().items()))

def openai_embedding(query):
    openai_client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
    response = openai_client.embeddings.create(
        model="text-embedding-3-large",
        input=query
    )
    return [response.data[0].embedding]

def bm25_embedding( query):
        if not isinstance(query, list):
            query = [query]
        sparse_embeddings = list(bm25_ef.encode_queries(query))[0]
        sparse_embeddings = [convert_sparse(sparse_embeddings)]
        return sparse_embeddings
    

start to install package: unidic-lite
successfully installed package: unidic-lite


You should consider upgrading via the '/Users/yj/.pyenv/versions/3.9.11/envs/myMechanic_py39/bin/python3.9 -m pip install --upgrade pip' command.


In [183]:
def sparse_search(query, sparse_params, topK=5):
    print("Sparse Search")
    sparse_search_params = {
        "anns_field": "bm25_vector",
        "param": {
            "metric_type": "IP",
            "params": sparse_params
        },
        "limit":topK,
        "expr": f'car_type == "{NAMESPACE}"'
    }
    
    sparse_params_wrapper = copy.deepcopy(sparse_search_params)

    data = bm25_embedding(query)
    if len(data[0]) == 0:
        print('No token for sparse search')
        return
    sparse_params_wrapper['data'] = data
        
    search_dict = {}
    res = collection.search(
        **sparse_params_wrapper,
        output_fields=[
            'id', 'majorheading', 'minorheading', 'minorheading_sub_id', 'parent_doc_id', 'doc_contents', 'img_urls', 'table_img_urls', 'table_csv_urls', 'embedding_contents']
    )

    for i, result in enumerate(res[0]):
        doc_entity = result.entity
        doc = {
            'distance': result.distance,
            'id': doc_entity.get('id'),
            'majorheading': doc_entity.get('majorheading'),
            'minorheading': doc_entity.get('minorheading'),
            'minorheading_sub_id': doc_entity.get('minorheading_sub_id'),
            'parent_doc_id': doc_entity.get('parent_doc_id'),
            'doc_contents': doc_entity.get('doc_contents'),
            'img_urls': doc_entity.get('img_urls'),
            'table_img_urls': doc_entity.get('table_img_urls'),
            'table_csv_urls': doc_entity.get('table_csv_urls'),
            'embedding_contents': doc_entity.get('embedding_contents')
        }
        search_dict[i] = doc
    return search_dict

def dense_search(query, dense_params, topK=5):
    print("Vector Search")
    dense_search_params = {
        "anns_field": "vector",
        "param": {
            "index_type": "HNSW",
            "metric_type": "COSINE",
            "params": dense_params
        }, # the ratio of small vector values to be dropped during search.
        "limit":topK,
        "expr": f'car_type == "{NAMESPACE}"'
    }
    
    dense_params_wrapper = copy.deepcopy(dense_search_params)

    dense_params_wrapper['data'] = openai_embedding(query)
        
    search_dict = {}
    res = collection.search(
        **dense_params_wrapper,
        output_fields=[
            'id', 'majorheading', 'minorheading', 'minorheading_sub_id', 'parent_doc_id', 'doc_contents', 'img_urls', 'table_img_urls', 'table_csv_urls', 'embedding_contents']
    )

    for i, result in enumerate(res[0]):
        doc_entity = result.entity
        doc = {
            'distance': result.distance,
            'id': doc_entity.get('id'),
            'majorheading': doc_entity.get('majorheading'),
            'minorheading': doc_entity.get('minorheading'),
            'minorheading_sub_id': doc_entity.get('minorheading_sub_id'),
            'parent_doc_id': doc_entity.get('parent_doc_id'),
            'doc_contents': doc_entity.get('doc_contents'),
            'img_urls': doc_entity.get('img_urls'),
            'table_img_urls': doc_entity.get('table_img_urls'),
            'table_csv_urls': doc_entity.get('table_csv_urls'),
            'embedding_contents': doc_entity.get('embedding_contents')
        }
        search_dict[i] = doc
    return search_dict

def hybrid_search(query, sparse_params, dense_params, rrk_method = 'weight', rerank_weight=(0.2, 0.8), topK=5):
    sparse_search_params = {
        "anns_field": "bm25_vector",
        "param": {
            "metric_type": "IP",
            "params": sparse_params
        },
        "limit":topK*2,
        "expr": f'car_type == "{NAMESPACE}"'
    }
    
    dense_search_params = {
        "anns_field": "vector",
        "param": {
            "index_type": "HNSW",
            "metric_type": "COSINE",
            "params": dense_params
        }, # the ratio of small vector values to be dropped during search.
        "limit":topK*2,
        "expr": f'car_type == "{NAMESPACE}"'
    }

    sparse_params_wrapper = copy.deepcopy(sparse_search_params)
    data = bm25_embedding(query)
    sparse_params_wrapper['data'] = data
    
    dense_params_wrapper = copy.deepcopy(dense_search_params)
    dense_params_wrapper['data'] = openai_embedding(query)  
    
    sparse_req = AnnSearchRequest(**sparse_params_wrapper)
    dense_req = AnnSearchRequest(**dense_params_wrapper)  
    
    
    is_empty_token = len(sparse_params_wrapper['data'][0]) == 0
    if is_empty_token:
        dense_params_wrapper['limit'] = topK
        print("Vector Search")
        reqs = dense_params
        # reqs['limit'] = int(reqs['limit']/2)
        res = collection.search(
            **dense_params_wrapper,
            #reqs, 
            output_fields=[
                'id', 'majorheading', 'minorheading', 'minorheading_sub_id', 'parent_doc_id', 'doc_contents', 'img_urls', 'table_img_urls', 'table_csv_urls', 'embedding_contents']
        )
    else:
        print("Hybrid Search")
        reqs = [sparse_req, dense_req]  
        if rrk_method == 'weight':
            rerank = WeightedRanker(*rerank_weight)
        else:
            rerank = RRFRanker()
        res = collection.hybrid_search(
            reqs, 
            rerank, 
            limit=topK,
            output_fields=[
                'id', 'majorheading', 'minorheading', 'minorheading_sub_id', 'parent_doc_id', 'doc_contents', 'img_urls', 'table_img_urls', 'table_csv_urls', 'embedding_contents'
                ]
        )
    search_dict = {}
    
    for i, result in enumerate(res[0]):
        doc_entity = result.entity
        doc = {
            'distance': result.distance,
            'id': doc_entity.get('id'),
            'majorheading': doc_entity.get('majorheading'),
            'minorheading': doc_entity.get('minorheading'),
            'minorheading_sub_id': doc_entity.get('minorheading_sub_id'),
            'parent_doc_id': doc_entity.get('parent_doc_id'),
            'doc_contents': doc_entity.get('doc_contents'),
            'img_urls': doc_entity.get('img_urls'),
            'table_img_urls': doc_entity.get('table_img_urls'),
            'table_csv_urls': doc_entity.get('table_csv_urls'),
            'embedding_contents': doc_entity.get('embedding_contents')
        }
        search_dict[i] = doc
    return search_dict

In [248]:
qa = pd.read_parquet('../../rag/qaset/qa_predict_new.parquet')
qa = qa[~qa['top3']][['doc_contents', 'question', 'doc_id', 'retreiver_1', 'retreiver_2', 'retreiver_3', 'retreiver_4', 'retreiver_5']].reset_index(drop=True)

context_file_root_path = '/Users/yj/Kim/1.work/SKR/8.GenAI/my-small-mechanic/pdf_context'
doc = pd.read_parquet(context_file_root_path+f'/embedding/{NAMESPACE}_embeddings.parquet')

In [242]:
qa.head(50)

,doc_contents,question,doc_id,retreiver_1,retreiver_2,retreiver_3,retreiver_4,retreiver_5
0,"\n미디어\n\n\n미디어 청취, 재생 기본 설정 조정:\n\n• ""[곡 이름] 듣...",미디어를 재생할 때 사용할 수 있는 명령어는 어떤 것들이 있나요?,2,26,20,19.0,21.0,NaN
1,"\n미디어\n\n\n미디어 청취, 재생 기본 설정 조정:\n\n• ""[곡 이름] 듣...",앱 및 설정을 탐색할 때 사용할 수 있는 명령어는 어떤 것들이 있나요?,2,26,7,13.0,30.0,203.0
2,[음성 명령 예]\n음성 명령 예\n\n음성 명령 예의 목록은 다음과 같습니다. 이...,실내 온도 조절 장치에서 어떤 설정을 조정할 수 있는가?,6,143,11,161.0,165.0,163.0
3,[음성 명령 예]\n음성 명령 예\n\n음성 명령 예의 목록은 다음과 같습니다. 이...,윈드실드 와이퍼에서 어떤 동작을 수행할 수 있는가?,6,7,372,278.0,271.0,208.0
4,[키 유형]\n키 유형\n\nModel 3 은(는) 다음과 같은 유형의 키를 지원합...,리모트키는 어떤 기능을 제공하나요?,8,217,216,243.0,352.0,20.0
5,\n터치스크린\n\n풀 셀프-드라이빙(장착된 경우)이 활성화되면 차량 상태 영역에 ...,주행 모드 스트립은 어떤 용도로 사용되나요?,11,340,182,349.0,0.0,337.0
6,[상단 상태 표시줄 아이콘]\n상단 상태 표시줄 아이콘\n\n터치하면 모든 도어와 ...,차량이 주차 상태일 때 어디에 Model 3이 표시되나요?,15,362,5,374.0,20.0,232.0
7,[상단 상태 표시줄 아이콘]\n상단 상태 표시줄 아이콘\n\n터치하면 모든 도어와 ...,감시 모드를 어떻게 수동으로 활성화하거나 비활성화할 수 있나요?,15,140,138,134.0,141.0,NaN
8,[상단 상태 표시줄 아이콘]\n상단 상태 표시줄 아이콘\n\n터치하면 모든 도어와 ...,감시 모드를 자동으로 켜려면 어떤 설정을 활성화해야 하나요?,15,141,134,138.0,140.0,139.0
9,"[개요]\n개요\n\n참고: 판매 지역, 제조 날짜 및 차량 구성에 따라 엔터테인먼...",USB 포트의 용도는 무엇인가요?,20,12,21,136.0,135.0,140.0


In [249]:
sparse_params = {"drop_ratio_search": 0}
dense_params = {"ef": 100}

idx = 44
doc_id, doc_contents, query = qa['doc_id'].values[idx], qa['doc_contents'].values[idx], qa['question'].values[idx]
query

'충전 포트 래치의 얼음을 녹이기 위해 어떤 기능을 사용해야 하나요?'

In [250]:
topK = 5
dense_result = dense_search(query, dense_params, topK)
sparse_result = sparse_search(query, sparse_params, topK)
weight_hybrid_result = hybrid_search(query, sparse_params=sparse_params, dense_params=dense_params, rrk_method='weight',rerank_weight=(0.43, 0.57), topK=topK)
rrf_hybrid_result = hybrid_search(query, sparse_params=sparse_params, dense_params=dense_params, rrk_method='rrf', topK=topK)

dense_docs = list(map(lambda x: x[1]['doc_contents'], dense_result.items()))
sparse_docs = list(map(lambda x: x[1]['doc_contents'], sparse_result.items()))
weight_hybrid_docs = list(map(lambda x: x[1]['doc_contents'], weight_hybrid_result.items()))
rrf_hybrid_docs = list(map(lambda x: x[1]['doc_contents'], rrf_hybrid_result.items()))
dense_docs = sorted(set(dense_docs), key=dense_docs.index)
sparse_docs = sorted(set(sparse_docs), key=sparse_docs.index)
weight_hybrid_docs = sorted(set(weight_hybrid_docs), key=weight_hybrid_docs.index)
rrf_hybrid_docs = sorted(set(rrf_hybrid_docs), key=rrf_hybrid_docs.index)

print(doc_id, ":", query)
print('Dense:', list(map(lambda x: x[1]['parent_doc_id'], dense_result.items())), list(map(lambda x: x[1]['distance'], dense_result.items())))
print('Sparse:', list(map(lambda x: x[1]['parent_doc_id'], sparse_result.items())), list(map(lambda x: x[1]['distance'], sparse_result.items())))
print('Weight Hybrid:', list(map(lambda x: x[1]['parent_doc_id'], weight_hybrid_result.items())), list(map(lambda x: x[1]['distance'], weight_hybrid_result.items())))
print('RRF Hybrid:', list(map(lambda x: x[1]['parent_doc_id'], rrf_hybrid_result.items())), list(map(lambda x: x[1]['distance'], rrf_hybrid_result.items())))


Vector Search
Sparse Search
Hybrid Search
Hybrid Search
70 : 충전 포트 래치의 얼음을 녹이기 위해 어떤 기능을 사용해야 하나요?
Dense: [71, 167, 70, 71, 66] [0.6035180687904358, 0.5561222434043884, 0.5325192213058472, 0.49866628646850586, 0.4897189736366272]
Sparse: [71, 70, 75, 388, 167] [20.996196746826172, 18.50360870361328, 18.302230834960938, 14.452695846557617, 13.857023239135742]
Weight Hybrid: [71, 70, 167, 75, 388] [0.8804461359977722, 0.8593885898590088, 0.8431278467178345, 0.841368556022644, 0.8386321067810059]
RRF Hybrid: [71, 70, 167, 75, 388] [0.032786883413791656, 0.0320020467042923, 0.03053613007068634, 0.030365770682692528, 0.02991071343421936]


In [251]:
print('정답 문서 ID와 질의: ', doc_id, ":", query)
print(doc_contents)

정답 문서 ID와 질의:  70 : 충전 포트 래치의 얼음을 녹이기 위해 어떤 기능을 사용해야 하나요?

CP_a078
케이블 차단됨 - 충전 포트 래치 결빙 가능모바일 앱에서 차량 성에 제거 버튼을 사용해 보십시오

이 경고의 의미:

충전 포트 래치에서 충전 케이블을 해제할 수 없고 차가운 주변 온도가 감지됩니다.

필요한 작업:

케이블을 느슨하게 하려면 충전 케이블을 충전 포트 유입구에 완전히 다시 삽입합니다. 충전 케이블 래치 해제를 시도해 봅니다.

충전 케이블을 여전히 탈거할 수 없는 경우 충전 포트 래치가 결빙되었을 수 있습니다.

충전 포트 래치의 얼음을 녹이는 데 도움이 되도록 Tesla 앱에서 차량 성에 제거 버튼을 눌러 약 30~45분간 차량의 성에를 제거합니다.

참고: 차량의 성에 제거를 위해 모바일 앱에서 차량 성에 제거를 사용하세요. 차량의 터치스크린에서 실내 온도 조절 설정을 조정하는 것은 그리 효과적이지 않습니다.

또한, 차량의 터치스크린을 통해 후면 성에 제거 기능을 켜서 충전 포트 래치에 영향을 주는 얼음을 녹일 수도 있습니다. 일부 차량에는 추운 날씨 조건에서 후면 성에 제거 기능을 켤 때 작동하는 충전 포트 인입구 히터가 장착되어 있습니다.

추운 날씨 조건에서의 충전에 관한 자세한 내용은 추운 날씨 모범 사례 페이지의 133을(를) 참조하세요.

충전 케이블을 여전히 탈거할 수 없는 경우, 차량 트렁크에 있는 충전 포트 수동 해제 케이블을 사용해 보세요.

1. 차량이 충전 중이 아닌지 확인하세요.◦ 차량 터치스크린에서 충전 화면에 액세스합니다.

◦ 필요한 경우 충전 중지를 터치합니다.

2. 후면 트렁크를 엽니다.

3. 충전 포트 해제 케이블을 아래로 당겨 충전 케이블 래치를 해제합니다.

◦ 참고: 해제 케이블은 후면 트렁크 왼쪽에 있습니다. 트렁크 실내 트림 소형 개방구 내부의 오목한 곳에 있을 수 있습니다.

4. 충전 케이블을 충전 포트에서 당깁니다.

충전 포트 수동 해제 사용에 관한 자세한 내용은 충전

In [252]:
for d in rrf_hybrid_docs:
    print(d)
    print('============================================================================================================================')


CP_a079
충전 속도 감소됨 - 충전 포트 결빙 가능모바일 앱에서 차량 성에 제거 버튼을 사용해 보십시오

이 경고의 의미:

충전 포트 래치가 충전 케이블을 충전 포트 유입구에 고정할 수 없고 차가운 주변 온도가 감지됩니다. 래치가 체결되지 않은 경우AC 충전(예: 모바일 커넥터 또는 월 커넥터로 충전)이 16A로 제한되고 DC 급속 충전/수퍼차징을 사용할 수 없습니다.

AC 충전 중 이 경고가 나타나면 충전 포트 표시등이 황색으로 점멸하고DC 급속 충전/수퍼차징 시도 시 이 경고가 나타나면 충전 포트 표시등이 황색으로 점등됩니다.

이 경고는 보통 외부 충전 장비 및 전력원 문제에 해당하며, 일반적으로 정비를 예약하여 해결할 수 있는 차량 문제를 나타내지 않습니다.

필요한 작업:

충전 포트 유입구에 충전 케이블을 다시 완전히 삽입해 봅니다. 차량 충전이 시작되고 충전 포트 표시등이 녹색으로 점멸하면 이전에 충전 포트가 완전히 삽입되지 않았을 수 있습니다. AC 충전이 더 이상 제한되지 않고 DC 급속 충전/수퍼차징을 사용할 수 있어야 합니다. 

충전이 여전히 제한되거나 차량이 전혀 충전되지 않는 경우 충전 포트 래치 수동 해제 케이블(트렁크의 왼쪽에 있음)이 당겨지지 않았는지 확인하십시오. 수동 해제 케이블의 핸들(일반적으로 링 형태 또는 스트랩)에 장애물이 없고 아무것도 부착되어 있지 않은지(예: 트렁크 네트 또는 우산 등) 확인하세요. 충전 포트 수동 해제 사용에 관한 자세한 내용은 충전 케이블 수동 해제 페이지의 154을(를) 참조하세요.

충전이 여전히 제한되거나 차량이 전혀 충전되지 않는 경우, 충전 포트 유입구 및 충전 케이블에 오염물, 수분 및/또는 이물질과 같은 장애물이 있는지 검사하십시오. 충전 포트 유입구 장애물이 제거되었고 수분이 건조되었는지 확인한 다음 케이블을 충전 포트에 다시 삽입해 봅니다.

파편 또는 이물질이 있는지 확인하여 모두 제거했지만 여전히 충전이 제한되거나 차량이 전혀 충전되지 않는 경우 충전 포트 래치가 

## BM25 토크나이커

In [233]:
relevant_doc = rrf_hybrid_docs[0]
relevant_doc = '모바일 커넥터'

In [234]:
query

'모바일 커넥터를 사용할 때 어떤 순서로 작업해야 하는가?'

In [235]:
query_embedding_result = convert_sparse(bm25_ef.encode_queries([query]))
doc_embedding_result = convert_sparse(bm25_ef.encode_documents([relevant_doc]))

print(query_embedding_result)
print(doc_embedding_result)

{16: 0.2680019320858422, 83: 3.1395305483516105, 214: 0.8834128963772825, 229: 0.8834128963772825, 1918: 5.841610180451529, 2210: 2.550486295433982, 6808: 5.841610180451529}
{229: 1.8042296760085001, 2071: 1.8042296760085001}
